In [1]:
# Landcover consensus

In [ ]:
using SpeciesDistributionToolkit
using CairoMakie
CairoMakie.activate!(px_per_unit = 6.0)

In [ ]:
place = SpeciesDistributionToolkit.gadm("PRY")
extent = SpeciesDistributionToolkit.boundingbox(place)

In [ ]:
provider = RasterData(EarthEnv, LandCover)
layers = SDMLayer{UInt8}[SDMLayer(provider; layer=l, extent...) for l in layers(provider)]

In [ ]:
consensus = mosaic(argmax, layers)

In [ ]:
heatmap(consensus)

In [ ]:
landcover_colors = [
    colorant"#117733",
    colorant"#668822",
    colorant"#99BB55",
    colorant"#55aa22",
    colorant"#ddcc66",
    colorant"#aaddcc",
    colorant"#44aa88",
    colorant"#88bbaa",
    colorant"#bb0011",
    :aqua,
    colorant"#FFEE88",
    colorant"#5566AA",
];

In [ ]:
fig = Figure(; size = (900, 1000))
panel = Axis(fig[1, 1]; aspect = DataAspect())
heatmap!(panel, consensus; colormap = landcover_colors)
Legend(
    fig[2, 1],
    [PolyElement(; color = landcover_colors[i]) for i in eachindex(landcover_colors)],
    landcover_types;
    orientation = :horizontal,
    nbanks = 4,
)
fig